In [ ]:
library(terra)
library(sf)
library(ggplot2)

process_raster_aoi <- function(df_lut, raster_path, aoi_path, region_name, epsg_code_tif = 4326, pixel_resolution = 100) {
  # Load the AOI polygon
  print("Loading AOI polygon...")
  aoi <- st_read(aoi_path)
  aoi <- st_transform(aoi, crs = epsg_code_tif)
  print("AOI polygon loaded and transformed.")
  
  # Load the raster file
  print("Loading raster file...")
  raster <- rast(raster_path)
  print("Raster file loaded.")
  
  # Mask the raster with the AOI
  print("Masking raster with AOI...")
  masked_raster <- mask(raster, vect(aoi))
  print("Raster masked with AOI.")
  
  # Get the values from the masked raster and filter out NA values
  print("Extracting values from masked raster...")
  values <- values(masked_raster)
  values <- values[!is.na(values)]
  print("Values extracted and filtered.")
  
  # Count the occurrences of each class
  print("Counting occurrences of each class...")
  class_distribution <- table(values)
  print("Class occurrences counted.")
  
  # Convert to a data frame and merge with df_lut
  print("Merging class distribution with lookup table...")
  class_distribution_df <- as.data.frame(class_distribution, stringsAsFactors = FALSE)
  colnames(class_distribution_df) <- c('Value', 'Count')
  result_df <- merge(class_distribution_df, df_lut, by = 'Value')
  print("Class distribution merged with lookup table.")
  
  # Calculate the area in square meters
  print("Calculating area in square kilometers...")
  result_df$Area_sq_km <- result_df$Count * pixel_resolution * pixel_resolution / 1000000
  
  # Calculate the percentage cover of total AOI for each class
  print("Calculating percentage cover for each class...")
  total_area <- sum(result_df$Area_sq_km)
  result_df$Percentage_Cover <- (result_df$Area_sq_km / total_area) * 100
  print("Percentage cover calculated.")
  
  print("Final result:")
  print(result_df[order(-result_df$Count), c('Description', 'Area_sq_km', 'Percentage_Cover', 'Count')])
  
  # Plot the results
  print("Plotting results...")
  ggplot() +
    geom_sf(data = aoi, fill = NA, color = 'red', size = 1) +
    geom_raster(data = as.data.frame(masked_raster, xy = TRUE), aes(x = x, y = y, fill = as.factor(values))) +
    scale_fill_manual(values = setNames(df_lut$Color, df_lut$Value)) +
    theme_minimal() +
    labs(title = paste(region_name, "Land use"), fill = "Land use class") +
    theme(legend.position = "right")
  print("Plotting complete.")
}

terra 1.7.83

Linking to GEOS 3.12.1, GDAL 3.8.4, PROJ 9.4.0; sf_use_s2() is TRUE



In [ ]:
# Example usage
df_lut <- data.frame(
  Description = c("Water", "Trees", "Flooded vegetation", "Crops", "Built Area", "Bare ground", "Snow/Ice", "Clouds", "Rangeland"),
  Value = c(1, 2, 4, 5, 7, 8, 9, 10, 11),
  Color = c("#1f77b4", "#2ca02c", "#17becf", "#ff7f0e", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22")
)

raster_path <- "../data/10m_lulc/SPAIN_2023_LULC.tif"
aoi_path <- "../data/aoi/AoI_Spain_3eData.geojson"
process_raster_aoi(df_lut, raster_path, aoi_path, "Spain", 32629, 10)

## Spain explore

In [ ]:
df_lut <- data.frame(
  Description = c("Water", "Trees", "Flooded vegetation", "Crops", "Built Area", "Bare ground", "Snow/Ice", "Clouds", "Rangeland"),
  Value = c(1, 2, 4, 5, 7, 8, 9, 10, 11),
  Color = c("#1f77b4", "#2ca02c", "#17becf", "#ff7f0e", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22")
)

raster_path <- "../data/10m_lulc/SPAIN_2023_LULC.tif"
aoi_path <- "../data/aoi/AoI_Spain_3eData.geojson"# Load the AOI polygon
aoi <- st_read(aoi_path)
aoi <- st_transform(aoi, crs = 32629)

Reading layer `AoI_Ancares_Courel' from data source 
  `/data/aoi/AoI_Spain_3eData.geojson' using driver `GeoJSON'
Simple feature collection with 1 feature and 1 field
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -7.508481 ymin: 42.35214 xmax: -6.690546 ymax: 43.12936
Geodetic CRS:  WGS 84


In [ ]:
# Load the raster file
print("Loading raster file...")
raster <- rast(raster_path)
print("Raster file loaded.")

summary(raster)

[1] "Loading raster file..."
[1] "Raster file loaded."


Warning message:
“[summary] used a sample”


 SPAIN_2023_LULC
 Min.   : 1.00  
 1st Qu.: 2.00  
 Median : 5.00  
 Mean   : 5.82  
 3rd Qu.:11.00  
 Max.   :11.00  
 NA's   :77897  

In [ ]:
# Mask the raster with the AOI
print("Masking raster with AOI...")
masked_raster <- mask(raster, vect(aoi))
print("Raster masked with AOI.")

In [ ]:
vect(aoi)

 class       : SpatVector 
 geometry    : polygons 
 dimensions  : 1, 1  (geometries, attributes)
 extent      : 622590.1, 688705.5, 4690252, 4777257  (xmin, xmax, ymin, ymax)
 coord. ref. : WGS 84 / UTM zone 29N (EPSG:32629) 
 names       :           NAME
 type        :          <chr>
 values      : Ancares_Courel

In [ ]:
crs(aoi)

[1] "PROJCRS[\"WGS 84 / UTM zone 29N\",\n    BASEGEOGCRS[\"WGS 84\",\n        ENSEMBLE[\"World Geodetic System 1984 ensemble\",\n            MEMBER[\"World Geodetic System 1984 (Transit)\"],\n            MEMBER[\"World Geodetic System 1984 (G730)\"],\n            MEMBER[\"World Geodetic System 1984 (G873)\"],\n            MEMBER[\"World Geodetic System 1984 (G1150)\"],\n            MEMBER[\"World Geodetic System 1984 (G1674)\"],\n            MEMBER[\"World Geodetic System 1984 (G1762)\"],\n            MEMBER[\"World Geodetic System 1984 (G2139)\"],\n            ELLIPSOID[\"WGS 84\",6378137,298.257223563,\n                LENGTHUNIT[\"metre\",1]],\n            ENSEMBLEACCURACY[2.0]],\n        PRIMEM[\"Greenwich\",0,\n            ANGLEUNIT[\"degree\",0.0174532925199433]],\n        ID[\"EPSG\",4326]],\n    CONVERSION[\"UTM zone 29N\",\n        METHOD[\"Transverse Mercator\",\n            ID[\"EPSG\",9807]],\n        PARAMETER[\"Latitude of natural origin\",0,\n            ANGLEUNIT[\"degree\",0.0174532925199433],\n            ID[\"EPSG\",8801]],\n        PARAMETER[\"Longitude of natural origin\",-9,\n            ANGLEUNIT[\"degree\",0.0174532925199433],\n            ID[\"EPSG\",8802]],\n        PARAMETER[\"Scale factor at natural origin\",0.9996,\n            SCALEUNIT[\"unity\",1],\n            ID[\"EPSG\",8805]],\n        PARAMETER[\"False easting\",500000,\n            LENGTHUNIT[\"metre\",1],\n            ID[\"EPSG\",8806]],\n        PARAMETER[\"False northing\",0,\n            LENGTHUNIT[\"metre\",1],\n            ID[\"EPSG\",8807]]],\n    CS[Cartesian,2],\n        AXIS[\"(E)\",east,\n            ORDER[1],\n            LENGTHUNIT[\"metre\",1]],\n        AXIS[\"(N)\",north,\n            ORDER[2],\n            LENGTHUNIT[\"metre\",1]],\n    USAGE[\n        SCOPE[\"Navigation and medium accuracy spatial referencing.\"],\n        AREA[\"Between 12°W and 6°W, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Côte D'Ivoire (Ivory Coast). Faroe Islands. Guinea. Ireland. Jan Mayen. Liberia, Mali. Mauritania. Morocco. Portugal. Sierra Leone. Spain. United Kingdom (UK). Western Sahara.\"],\n        BBOX[0,-12.01,84.01,-6]],\n    ID[\"EPSG\",32629]]"

## Zambia explore

In [ ]:
df_lut <- data.frame(
  Description = c("Water", "Trees", "Flooded vegetation", "Crops", "Built Area", "Bare ground", "Snow/Ice", "Clouds", "Rangeland"),
  Value = c(1, 2, 4, 5, 7, 8, 9, 10, 11),
  Color = c("#1f77b4", "#2ca02c", "#17becf", "#ff7f0e", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22")
)

raster_path <- "../data/10m_lulc/ZAMBIA_2023_35L_20230101-20240101.tif"
aoi_path <- "../data/aoi/AoI_Zambia_By_Life_Connected.geojson"
# Load the AOI polygon
aoi <- st_read(aoi_path)
aoi <- st_transform(aoi, crs = 32735)

Reading layer `AoI_Mponda' from data source 
  `/data/aoi/AoI_Zambia_By_Life_Connected.geojson' using driver `GeoJSON'
Simple feature collection with 1 feature and 1 field
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 26.77027 ymin: -15.1321 xmax: 26.92342 ymax: -14.85555
Geodetic CRS:  WGS 84


In [ ]:
# Load the raster file
print("Loading raster file...")
raster <- rast(raster_path)
print("Raster file loaded.")

summary(raster)

[1] "Loading raster file..."
[1] "Raster file loaded."


Warning message:
“[summary] used a sample”


 ZAMBIA_2023_35L_20230101.20240101
 Min.   : 1.000                   
 1st Qu.: 2.000                   
 Median : 2.000                   
 Mean   : 4.991                   
 3rd Qu.:11.000                   
 Max.   :11.000                   
 NA's   :1480                     

In [ ]:
# Mask the raster with the AOI
print("Masking raster with AOI...")
masked_raster <- mask(raster, vect(aoi))
print("Raster masked with AOI.")

[1] "Masking raster with AOI..."
[1] "Raster masked with AOI."             


In [ ]:
# Get the values from the masked raster and filter out NA values
print("Extracting values from masked raster...")
values <- values(masked_raster)
values <- values[!is.na(values)]
print("Values extracted and filtered.")

# Count the occurrences of each class
print("Counting occurrences of each class...")
class_distribution <- table(values)
print("Class occurrences counted.")

# Convert to a data frame and merge with df_lut
print("Merging class distribution with lookup table...")
class_distribution_df <- as.data.frame(class_distribution, stringsAsFactors = FALSE)
colnames(class_distribution_df) <- c('Value', 'Count')
result_df <- merge(class_distribution_df, df_lut, by = 'Value')
print("Class distribution merged with lookup table.")

# Calculate the area in square meters
print("Calculating area in square kilometers...")
result_df$Area_sq_km <- result_df$Count * pixel_resolution * pixel_resolution / 1000000

# Calculate the percentage cover of total AOI for each class
print("Calculating percentage cover for each class...")
total_area <- sum(result_df$Area_sq_km)
result_df$Percentage_Cover <- (result_df$Area_sq_km / total_area) * 100
print("Percentage cover calculated.")

print("Final result:")
print(result_df[order(-result_df$Count), c('Description', 'Area_sq_km', 'Percentage_Cover', 'Count')])

[1] "Extracting values from masked raster..."
